In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=471e083199d20082d868fb0061562ebfd1bef1e45772e6337f0c4960f08da462
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [1]:
from tensorflow import keras
from keras.datasets import fashion_mnist
from keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt
import random
import wandb
import argparse
from datetime import datetime
import copy

class FeedForward:

    def __init__(self):

        # defining the default parameters
        self.parameters = {
            "wandb_project": "DL Final Assignment 1",
            "wandb_entity": "cs22m019",
            "dataset": "fashion_mnist",
            "epochs": 5,
            "batch_size": 32,
            "loss": "mean_squared_error",
            "optimizer": "gd",
            "learning_rate": 0.1,
            "momentum": 0.01,
            "beta": 0.5,
            "beta1": 0.5,
            "beta2": 0.5,
            "epsilon": 0.000001,
            "weight_decay": 0.0,
            "weight_init": "random",
            "num_layers": 3,
            "hidden_size": 128,
            "activation": "sigmoid",
            "output_function": "softmax"
        }

        # updating the parameters to the parameters given in command line
        # self.update_parameters()

       
        # loading training and test data from fashion_mnist dataset or mnist dataset
        if (self.parameters["dataset"] == "fashion_mnist"):
            (self.x_train, self.y_train), (self.x_test,self.y_test) = fashion_mnist.load_data()
        else:
            (self.x_train, self.y_train), (self.x_test,self.y_test) = mnist.load_data()

        # normalizing data points
        self.x_train = self.x_train / 255
        self.x_test = self.x_test / 255

        # computing number of samples in training and test data
        self.train_n_samples = self.x_train.shape[0]
        self.test_n_samples = self.x_test.shape[0]

        # spiltting the data -> 90% train,10% test 
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        self.x_validate = self.x_train[: self.train_n_samples // 10]
        self.y_validate = self.y_train[: self.train_n_samples // 10]

        self.x_train = self.x_train[self.train_n_samples // 10:]
        self.y_train = self.y_train[self.train_n_samples // 10:]

        self.train_n_samples = self.x_train.shape[0]

        # list of label titles -> actual output
        self.title = ["T-shirt/top", "Trouser", "PullOver", "Dress",
                      "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
        self.no_of_label = len(self.title)

        # setting the class variables
        self.epoch = self.parameters["epochs"]
        self.batch_size = self.parameters["batch_size"]
        self.lossFunction = self.parameters["loss"]
        self.optimizer = self.parameters["optimizer"]
        self.learningRate = self.parameters["learning_rate"]
        self.weightInitialization = self.parameters["weight_init"]
        self.L = self.parameters["num_layers"] + 1
        self.hl = self.parameters["num_layers"]
        self.nnl = self.parameters["hidden_size"]
        self.activationFunction = self.parameters["activation"]
        self.outputFunction = self.parameters["output_function"]
        self.weight_decay = self.parameters["weight_decay"]
        
        
        
        self.k = len(self.title)
        self.d = self.x_train.shape[1] * self.x_train.shape[2]
        self.n = self.train_n_samples
        self.weights = {}
        self.bias = {}
        self.wHistory = {}
        self.bHistory = {}
        self.wMomentum = {}
        self.bMomentum = {}
        self.prev_wHistory = {}
        self.prev_bHistory = {}
        self.pre_activation = {}
        self.post_activation = {}
        
    # updates the default parameters with the paramters given in command line
    def update_parameters(self):

        parser = argparse.ArgumentParser(description='DL Assignment 1 Parser')

        parser.add_argument('-wp', '--wandb_project',
                            type=str, metavar='', help='wandb project')
        parser.add_argument('-we', '--wandb_entity', type=str,
                            metavar='', help='wandb entity')
        parser.add_argument('-d', '--dataset', type=str,
                            metavar='', help='dataset')
        parser.add_argument('-e', '--epochs', type=int,
                            metavar='', help='epochs')
        parser.add_argument('-b', '--batch_size', type=int,
                            metavar='', help='batch size')
        parser.add_argument('-l', '--loss', type=str, 
                            metavar='', help='loss')
        parser.add_argument('-o', '--optimizer', type=str,
                            metavar='', help='optimizer')
        parser.add_argument('-lr', '--learning_rate',
                            type=float, metavar='', help='learning rate')
        parser.add_argument('-m', '--momentum', type=float,
                            metavar='', help='momentum')
        parser.add_argument('-beta', '--beta', type=float,
                            metavar='', help='beta')
        parser.add_argument('-beta1', '--beta1', type=float,
                            metavar='', help='beta1')
        parser.add_argument('-beta2', '--beta2', type=float,
                            metavar='', help='beta2')
        parser.add_argument('-eps', '--epsilon', type=float,
                            metavar='', help='epsilon')
        parser.add_argument('-w_d', '--weight_decay',
                            type=float, metavar='', help='weight decay')
        parser.add_argument('-w_i', '--weight_init', type=str,
                            metavar='', help='weight init')
        parser.add_argument('-nhl', '--num_layers', type=int,
                            metavar='', help='num layers')
        parser.add_argument('-sz', '--hidden_size', type=int,
                            metavar='', help='hidden size')
        parser.add_argument('-a', '--activation', type=str,
                            metavar='', help='activation')
        parser.add_argument('-of', '--output_function',
                            type=str, metavar='', help='output function')
        args = parser.parse_args()

        if (args.wandb_project != None):
            self.parameters["wandb_project"] = args.wandb_project
        if (args.wandb_entity != None):
            self.parameters["wandb_entity"] = args.wandb_entity
        if (args.dataset != None):
            self.parameters["dataset"] = args.dataset
        if (args.epochs != None):
            self.parameters["epochs"] = args.epochs
        if (args.batch_size != None):
            self.parameters["batch_size"] = args.batch_size
        if (args.loss != None):
            self.parameters["loss"] = args.loss
        if (args.optimizer != None):
            self.parameters["optimizer"] = args.optimizer
        if (args.learning_rate != None):
            self.parameters["learning_rate"] = args.learning_rate
        if (args.momentum != None):
            self.parameters["momentum"] = args.momentum
        if (args.beta != None):
            self.parameters["beta"] = args.beta
        if (args.beta1 != None):
            self.parameters["beta1"] = args.beta1
        if (args.beta2 != None):
            self.parameters["beta2"] = args.beta2
        if (args.epsilon != None):
            self.parameters["epsilon"] = args.epsilon
        if (args.weight_decay != None):
            self.parameters["weight_decay"] = args.weight_decay
        if (args.weight_init != None):
            self.parameters["weight_init"] = args.weight_init
        if (args.num_layers != None):
            self.parameters["num_layers"] = args.num_layers
        if (args.hidden_size != None):
            self.parameters["hidden_size"] = args.hidden_size
        if (args.activation != None):
            self.parameters["activation"] = args.activation

    # function to initialize weights and bias based on type -> random or Xavier initialization
    def weightsAndBiasInitializer(self):
        if self.weightInitialization == "Xavier":

            # first and last matrix 
            self.weights["w1"] = np.random.uniform(-np.sqrt(6 / (self.nnl + self.d)), np.sqrt(6 / (self.nnl + self.d)), (self.nnl, self.d))
            self.weights["w" + str(self.L)] = np.random.uniform(-np.sqrt(6 / (self.k + self.nnl)),np.sqrt(6 / (self.k + self.nnl)), (self.k, self.nnl))

            # Intermediate Matrices
            for i in range(2, self.L):
                self.weights["w" + str(i)] = np.random.uniform(-np.sqrt(6 / (self.nnl + self.nnl)), np.sqrt(6 / (self.nnl + self.nnl)), (self.nnl, self.nnl))

            # Last Vector
            self.bias["b" + str(self.L)] = np.random.uniform(-np.sqrt(6 / (self.k + 1)),np.sqrt(6 / (self.k + 1)), (self.k))

            for i in range(1, self.L):
                self.bias["b" + str(i)] = np.random.uniform(-np.sqrt(6 / (self.nnl + 1)),np.sqrt(6 / (self.nnl + 1)), (self.nnl))

        if self.weightInitialization == "random":
            # initailzation of weights
            '''
                  W1 = (d,nnl)
                  W2,..,W(L - 1) = (nnl,nnl)
                  WL = (k,nnl)
            '''
            w1 = np.random.normal(0, 0.5, size=(self.nnl, self.d))
            self.weights["w1"] = w1
            for i in range(2, self.L):
                self.weights["w" + str(i)] = np.random.normal(0,0.5, size=(self.nnl, self.nnl))
            self.weights["w" + str(self.L)] = np.random.normal(0,0.5, size=(self.k, self.nnl))

            # initialization of bias
            for i in range(1, self.L):
                self.bias["b" + str(i)] = np.random.normal(0,0.5, size=(self.nnl))
            self.bias["b" + str(self.L)] = np.random.normal(0,0.5, size=(self.k))

    # function to initialize momentum for weights and bias
    def momentumInitializer(self):

        # initializing momentum for weights
        w1 = np.zeros((self.nnl, self.d))
        self.wMomentum["w1"] = w1
        for i in range(2, self.L):
            self.wMomentum["w" + str(i)] = np.zeros((self.nnl, self.nnl))
        self.wMomentum["w" + str(self.L)] = np.zeros((self.k, self.nnl))

        # initializing momentum for bais
        for i in range(1, self.L):
            self.bMomentum["b" + str(i)] = np.zeros((self.nnl))
        self.bMomentum["b" + str(self.L)] = np.zeros((self.k))

    # function to initialize history for weights and bias
    def historyInitializer(self):

        # initializing history for weights
        w1 = np.zeros((self.nnl, self.d))
        self.wHistory["w1"] = w1
        for i in range(2, self.L):
            self.wHistory["w" + str(i)] = np.zeros((self.nnl, self.nnl))
        self.wHistory["w" + str(self.L)] = np.zeros((self.k, self.nnl))

        # initializing history for bais
        for i in range(1, self.L):
            self.bHistory["b" + str(i)] = np.zeros((self.nnl))
        self.bHistory["b" + str(self.L)] = np.zeros((self.k))

    # function used to implement different activation functions
    def activation_func(self, vector):
        if self.activationFunction == "sigmoid":
            return 1.0 / (1 + np.exp(-(vector)))
        
        if self.activationFunction == "tanh":
            return np.tanh(vector)
        
        if self.activationFunction == "ReLU":
            return np.maximum(0,vector)

    # function used to implement different output functions
    def output_func(self, vector):
        if self.outputFunction == "softmax":

            vector = vector - vector[np.argmax(vector)]

            return np.exp(vector) / np.sum(np.exp(vector))

    # function generating one-hot vector
    def oneHotVector(self, size, index):
        oneHot = np.zeros(size)
        oneHot[index] = 1.0
        return oneHot

    # function returning the differentiation of activation function
    def differentiation(self, vector):

        if self.activationFunction == "sigmoid":
            return (1.0 / (1 + np.exp(-(vector)))) * (1 - 1.0 / (1 + np.exp(-(vector))))

        if self.activationFunction == "tanh":
            return 1 - (np.tanh(vector)) ** 2

        if self.activationFunction == "ReLU":
            t = np.maximum(0,vector)
            t[t > 0] = 1
            return t

    # regularization
    def regularize(self):
        reg_term = 0
        validation_size = self.y_validate.shape[0]
        
        for (key,value) in self.weights.items():
          reg_term += (np.sum(self.weights[key] ** 2))
        reg_term = (self.weight_decay / (2 * validation_size)) * reg_term

        return reg_term

    # function returning the loss function value
    def loss_function(self, y_predicted, index):

        if self.lossFunction == "cross_entropy":
            t = 1e-8
            return (-1)*np.log(y_predicted[index] + t)

        if self.lossFunction == "mean_squared_error":
            y = self.oneHotVector(size=self.no_of_label, index=index)
            return np.sum((y_predicted - y) ** 2)

    # forward propagation - computes pre_activation vector,post_activation vector for each layer and predicts y at last layer
    def forward_propagation(self, input, index):

        # Populating pre_activation and post_activation vectors to dictionary in each layer for input[index]
        for k in range(1, self.L):

            # for first layer,post activation will be input
            if (k == 1):
                ''' flattening the input: 
                    -input(60000,28,28)
                    -input[index] size = (28,28)
                    -flattening input[index] gives size (784,1) = (d,1) where d is dimension of input
                    post_activation[h0] size = (d,1)
                    bias[b1] size = (nnl,1)
                    weights[w1] size = (nnl,d)
                    Therefore we get pre_activation[a1] size = (nnl,1) for all layer except last layer
                '''
                self.post_activation["h" + str(k - 1)] = input[index].flatten()

            # computing a(k) = b(k) + w(k)*h(k - 1) for each input[index]
            self.pre_activation["a" + str(k)] = self.bias["b" + str(k)] + np.dot(self.weights["w" + str(k)], self.post_activation["h" + str(k - 1)])
           
            # computing h(k) = g(a(k)) where g is activation function
            self.post_activation["h" + str(k)] = self.activation_func(self.pre_activation["a" + str(k)])

        # computing pre_activation for last layer
        self.pre_activation["a" + str(self.L)] = self.bias["b" + str(self.L)] + np.dot(self.weights["w" + str(self.L)], self.post_activation["h" + str(self.L - 1)])

        # prediction y (y_hat) = O(a(L)) where O is output function
        self.post_activation["h" +str(self.L)] = self.output_func(self.pre_activation["a" + str(self.L)])

    # performs back propagation and returns gradients of weights and bias
    def backward_propagation(self, index, actual_y):

        grad_pre_activation = {}
        grad_post_activation = {}
        grad_weights = {}
        grad_bias = {}

        predicted_y = self.post_activation["h" + str(self.L)]

        # Computing output gradient
        one_hot_vector = self.oneHotVector(self.no_of_label, actual_y[index])
        if self.lossFunction == "cross_entropy" :
          grad_pre_activation["a" + str(self.L)] = (predicted_y - one_hot_vector)
        else :
          grad_pre_activation["a" + str(self.L)] = -2 * (one_hot_vector - predicted_y) * (predicted_y * (np.ones(self.no_of_label) - predicted_y))
       
        
        k = self.L
        while k > 0:

            # Computing gradient w.r.t parameters - weight and bais
            '''
              np.reshape(grad_pre_activation["a" + str(L)],(-1,1)) = (k,1)
              np.reshape(post_activation["h" + str(L - 1)],(1,-1)) = (1,nnl)
            '''
            grad_weights["w" + str(k)] = np.dot(np.reshape(grad_pre_activation["a" + str(k)], (-1, 1)), np.reshape(self.post_activation["h" + str(k - 1)], (1, -1)))
            grad_bias["b" + str(k)] = grad_pre_activation["a" + str(k)]

            if k != 1:
                # Computing gradient w.r.t layer below (post_activation)
                grad_post_activation["h" + str(k - 1)] = np.dot(self.weights["w" + str(k)].T, np.reshape(grad_pre_activation["a" + str(k)], (-1, 1))).flatten()

                # Computing gradient w.r.t layer below (pre_activation)
                g_dash = self.differentiation(self.pre_activation["a" + str(k - 1)])
                grad_pre_activation["a" +str(k - 1)] = grad_post_activation["h" + str(k - 1)] * g_dash

            k = k - 1
        return grad_weights, grad_bias

    # function to make accumalated gradients zero
    def make_accumalate_zero(self):

        acc_grad_weights = {}
        acc_grad_bias = {}

        # accumalated weights are set to zero
        acc_grad_weights["w1"] = np.zeros((self.nnl, self.d))
        for i in range(2, self.L):
            acc_grad_weights["w" + str(i)] = np.zeros((self.nnl, self.nnl))
        acc_grad_weights["w" + str(self.L)] = np.zeros((self.k, self.nnl))

        # accumalated bias are set to zero
        for i in range(1, self.L):
            acc_grad_bias["b" + str(i)] = np.zeros((self.nnl))
        acc_grad_bias["b" + str(self.L)] = np.zeros((self.k))

        return acc_grad_weights, acc_grad_bias

    # runs stochastic gradient descent for one epoch
    def oneEpochSGD(self, epoch):
        ''' Executes A Single Epoch for Stochastic Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''
        
        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # total Loss for epoch
        loss_input = 0
        count = 0

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perform forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # compute loss
            loss_input += self.loss_function(predicted_y, actual_y[index])

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(index, actual_y)

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):
                # update weights
                for (key, value) in self.weights.items():
                    self.weights[key] = self.weights[key] - ((self.learningRate / self.batch_size) * grad_weights[key])

                # update bias
                for (key, value) in self.bias.items():
                    self.bias[key] = self.bias[key] - ((self.learningRate / self.batch_size) * grad_bias[key])

        # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:
            # update weights
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] - ((self.learningRate / self.batch_size) * grad_weights[key])

            # update bias
            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - ((self.learningRate / self.batch_size) * grad_bias[key])

        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()

        # compute validationAccuracy,validationLoss avergaed over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs momentum gradient descent for one epoch
    def oneEpochMOMENTUM(self, epoch):
        ''' Executes A Single Epoch for Momentum Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''

        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # maintaining previous history for weights and bias
        self.prev_wHistory, self.prev_bHistory = self.wHistory, self.bHistory

        # Total Loss for epoch
        loss_input = 0
        count = 0  
        beta = self.parameters["momentum"]

        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perform forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # compute loss
            loss_input += self.loss_function(predicted_y, actual_y[index])

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(
                index, actual_y)

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]


            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):

                # update weight history
                for (key, value) in self.wHistory.items():
                    self.wHistory[key] = beta * self.prev_wHistory[key] + \
                        ((self.learningRate / self.batch_size) * acc_grad_weights[key])

                # update bias history
                for (key, value) in self.bHistory.items():
                    self.bHistory[key] = beta * self.prev_bHistory[key] + \
                        ((self.learningRate / self.batch_size) * acc_grad_bias[key])

                # update weights
                for (key, value) in self.weights.items():
                    self.weights[key] = self.weights[key] - self.wHistory[key]

                # update bias
                for (key, value) in self.bias.items():
                    self.bias[key] = self.bias[key] - self.bHistory[key]

                # updating histroy for weights and bias
                self.prev_wHistory = self.wHistory
                self.prev_bHistory = self.bHistory

                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

            # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:
            
            # update weight history
            for (key, value) in self.wHistory.items():
                self.wHistory[key] = beta * self.prev_wHistory[key] + \
                    ((self.learningRate / self.batch_size) * acc_grad_weights[key])

            # update bias history
            for (key, value) in self.bHistory.items():
                self.bHistory[key] = beta * self.prev_bHistory[key] + \
                    ((self.learningRate / self.batch_size) * acc_grad_bias[key])

            # update weights
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] - self.wHistory[key]

            # update bias
            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - self.bHistory[key]

            # updating histroy for weights and bias
            self.prev_wHistory = self.wHistory
            self.prev_bHistory = self.bHistory

            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()

        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs nestrov accelerated gradient descent for one epoch
    def oneEpochNAG(self, epoch):
        ''' Executes A Single Epoch for Nesterov Accelerated Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''
        
        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # maintaining previous history for weights and bias
        self.prev_wHistory, self.prev_bHistory = self.wHistory, self.bHistory
        
        # Total Loss for epoch
        loss_input = 0
        count = 0
        beta = self.parameters["momentum"]

        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
        
        # computing partial values
        self.partial_wHistory = {}
        self.partial_bHistory = {}

        for (key, value) in self.wHistory.items():
            self.partial_wHistory[key] = beta * self.prev_wHistory[key]

        for (key, value) in self.bHistory.items():
            self.partial_bHistory[key] = beta * self.prev_bHistory[key]

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perfrom forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # storing weights and bias in temperory values
            temp_weights = copy.deepcopy(self.weights)
            temp_bias = copy.deepcopy(self.bias)

            # update weights and bias
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] -  self.partial_wHistory[key]

            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - self.partial_bHistory[key]

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(index,actual_y)

            # update weights and bias 
            self.weights = temp_weights
            self.bias = temp_bias

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]

            # compute loss
            loss_input += self.loss_function(predicted_y, self.y_train[index])

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):
    
                # update weight history
                for (key, value) in self.wHistory.items():
                    self.wHistory[key] = beta * self.prev_wHistory[key] + ((self.learningRate / self.batch_size) * acc_grad_weights[key])

                # update bias history
                for (key, value) in self.bHistory.items():
                    self.bHistory[key] = beta * self.prev_bHistory[key] + ((self.learningRate / self.batch_size) * acc_grad_bias[key])

                # update weights
                for (key, value) in self.weights.items():
                    self.weights[key] = self.weights[key] - self.wHistory[key]

                for (key, value) in self.bias.items():
                    self.bias[key] = self.bias[key] - self.bHistory[key]

                # updating histroy for weights and bias
                self.prev_wHistory = self.wHistory
                self.prev_bHistory = self.bHistory
                
                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
            
        # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:

            # update weight history
            for (key, value) in self.wHistory.items():
                self.wHistory[key] = beta * self.prev_wHistory[key] + ((self.learningRate / self.batch_size) * acc_grad_weights[key])

            # update bias history
            for (key, value) in self.bHistory.items():
                self.bHistory[key] = beta * self.prev_bHistory[key] + ((self.learningRate / self.batch_size) * acc_grad_bias[key])

            # update weights
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] - self.wHistory[key]

            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - self.bHistory[key]

            # updating histroy for weights and bias
            self.prev_wHistory = self.wHistory
            self.prev_bHistory = self.bHistory
            
            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
                
        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()

        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs RMSPROP for one epoch
    def oneEpochRMSPROP(self,epoch):
        ''' Executes A Single Epoch for RMSPROP Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. 
        '''
        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train


        # maintaining previous history for weights and bias
        prev_wHistory, prev_bHistory = self.wHistory, self.bHistory

        # Total Loss for epoch
        loss_input = 0
        beta = self.parameters["beta"]
        eps = self.parameters["epsilon"]
        count = 0
        
        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perfrom forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(index,actual_y)

            # compute loss
            loss_input += self.loss_function(predicted_y, self.y_train[index])

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]
        

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):
    
                # update weight history
                for (key, value) in self.wHistory.items():
                    self.wHistory[key] = beta * prev_wHistory[key] +  (1 - beta) * acc_grad_weights[key] ** 2

                # update bias history
                for (key, value) in self.bHistory.items():
                    self.bHistory[key] = beta * prev_bHistory[key] + (1 - beta) * acc_grad_bias[key] ** 2

                # update weights
                for (key, value) in self.weights.items():
                    self.weights[key] = self.weights[key] -  (self.learningRate / self.batch_size) * acc_grad_weights[key] / (np.sqrt(self.wHistory[key] + eps))

                # update bias
                for (key, value) in self.bias.items():
                    self.bias[key] = self.bias[key] - (self.learningRate / self.batch_size) * acc_grad_bias[key] / (np.sqrt(self.bHistory[key] + eps))
                
                # updating histroy for weights and bias
                self.prev_wHistory = self.wHistory
                self.prev_bHistory = self.bHistory

                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:
            
            # update weight history
            for (key, value) in self.wHistory.items():
                self.wHistory[key] = beta * prev_wHistory[key] +  (1 - beta) * acc_grad_weights[key] ** 2

            # update bias history
            for (key, value) in self.bHistory.items():
                self.bHistory[key] = beta * prev_bHistory[key] + (1 - beta) * acc_grad_bias[key] ** 2

            # update weights
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] -  (self.learningRate / self.batch_size) * acc_grad_weights[key] / (np.sqrt(self.wHistory[key] + eps))

            # update bias
            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - (self.learningRate / self.batch_size) * acc_grad_bias[key] / (np.sqrt(self.bHistory[key] + eps))
            
            # updating histroy for weights and bias
            self.prev_wHistory = self.wHistory
            self.prev_bHistory = self.bHistory

            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()

        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs ADAM gradient descent for one epoch
    def oneEpochADAM(self,epoch):
        ''' Executes A Single Epoch for ADAM Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''

        n = self.train_n_samples
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # maintaining previous history and momentum for weights and bias
        prev_wMomentum, prev_bMomentum = self.wMomentum, self.bMomentum
        prev_wHistory, prev_bHistory = self.wHistory, self.bHistory

        wMomentum_hat = {} 
        bMomentum_hat = {}
        wHistory_hat = {} 
        bHistory_hat = {}

        # Total Loss for epoch
        loss_input = 0
        count = 0
        beta1 = self.parameters["beta1"]
        beta2 = self.parameters["beta2"]
        epsilon = self.parameters["epsilon"]
        
        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perform forward propagation

            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(index,actual_y)

            # compute loss
            loss_input += self.loss_function(predicted_y, actual_y[index])

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):
        
                # update weight momentum
                for (key, value) in self.wMomentum.items():
                    self.wMomentum[key] = beta1*prev_wMomentum[key] +  (1 - beta1) * acc_grad_weights[key]

                # update bias momentum
                for (key, value) in self.bMomentum.items():
                    self.bMomentum[key] = beta1*prev_bMomentum[key] + (1 - beta1) * acc_grad_bias[key]

                # update weight history
                for (key, value) in self.wHistory.items():
                    self.wHistory[key] = beta2 * prev_wHistory[key] + (1 - beta2) * acc_grad_weights[key] ** 2

                # update bias history
                for (key, value) in self.bHistory.items():
                    self.bHistory[key] = beta2 * prev_bHistory[key] + (1 - beta2) * acc_grad_bias[key] ** 2

                
                # compute intermediate values
                for (key, value) in self.weights.items():
                    wMomentum_hat[key] = self.wMomentum[key] / (1 - np.power(beta1, epoch + 1))

                for (key, value) in self.bias.items():
                    bMomentum_hat[key] = self.bMomentum[key] / (1 - np.power(beta1, epoch + 1))

                for (key, value) in self.weights.items():
                    wHistory_hat[key] = self.wHistory[key] / (1 - np.power(beta2, epoch + 1))

                for (key, value) in self.bias.items():
                    bHistory_hat[key] = self.bHistory[key] / (1 - np.power(beta2, epoch + 1))

                # update weights
                for (key, value) in self.weights.items():
                    temp = (self.learningRate / self.batch_size) * wMomentum_hat[key] / (np.sqrt(wHistory_hat[key] + epsilon))
                    self.weights[key] = self.weights[key] - temp
                
                # update bias
                for (key, value) in self.bias.items():
                    temp = (self.learningRate / self.batch_size) * bMomentum_hat[key] / (np.sqrt(bHistory_hat[key] + epsilon))
                    self.bias[key] = self.bias[key] - temp

                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
            
        # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:
            
            # update weight momentum
            for (key, value) in self.wMomentum.items():
                self.wMomentum[key] = beta1*prev_wMomentum[key] +  (1 - beta1) * acc_grad_weights[key]

            # update bias momentum
            for (key, value) in self.bMomentum.items():
                self.bMomentum[key] = beta1*prev_bMomentum[key] + (1 - beta1) * acc_grad_bias[key]

            # update weight history
            for (key, value) in self.wHistory.items():
                self.wHistory[key] = beta2 * prev_wHistory[key] + (1 - beta2) * acc_grad_weights[key] ** 2

            # update bias history
            for (key, value) in self.bHistory.items():
                self.bHistory[key] = beta2 * prev_bHistory[key] + (1 - beta2) * acc_grad_bias[key] ** 2

            
            # compute intermediate values
            for (key, value) in self.weights.items():
                wMomentum_hat[key] = self.wMomentum[key] / (1 - np.power(beta1, epoch + 1))

            for (key, value) in self.bias.items():
                bMomentum_hat[key] = self.bMomentum[key] / (1 - np.power(beta1, epoch + 1))

            for (key, value) in self.weights.items():
                wHistory_hat[key] = self.wHistory[key] / (1 - np.power(beta2, epoch + 1))

            for (key, value) in self.bias.items():
                bHistory_hat[key] = self.bHistory[key] / (1 - np.power(beta2, epoch + 1))

            # update weights
            for (key, value) in self.weights.items():
                temp = (self.learningRate / self.batch_size) * wMomentum_hat[key] / (np.sqrt(wHistory_hat[key] + epsilon))
                self.weights[key] = self.weights[key] - temp
            
            # update bias
            for (key, value) in self.bias.items():
                temp = (self.learningRate / self.batch_size) * bMomentum_hat[key] / (np.sqrt(bHistory_hat[key] + epsilon))
                self.bias[key] = self.bias[key] - temp

            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
        
        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()
        
        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs NADAM gradient descent for one epoch
    def oneEpochNADAM(self,epoch):
        ''' Executes A Single Epoch for NADAM Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''

        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # maintaining previous history and momentum for weights and bias
        prev_wMomentum, prev_bMomentum = self.wMomentum, self.bMomentum
        prev_wHistory, prev_bHistory = self.wHistory, self.bHistory

        wMomentum_hat = {} 
        bMomentum_hat = {}
        wHistory_hat = {} 
        bHistory_hat = {}

        # Total Loss for epoch
        loss_input = 0
        count = 0
        beta1 = self.parameters["beta1"]
        beta2 = self.parameters["beta2"]
        epsilon = self.parameters["epsilon"]


        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        for index in range(n):

            # perform forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # perform backward propagation
            grad_weights, grad_bias = self.backward_propagation(index,actual_y)

            # compute loss
            loss_input += self.loss_function(predicted_y, actual_y[index])

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):
                
                # update weight momentum
                for (key, value) in self.wMomentum.items():
                    self.wMomentum[key] = beta1*prev_wMomentum[key] + (1 - beta1) * acc_grad_weights[key]

                # update bias momentum
                for (key, value) in self.bMomentum.items():
                    self.bMomentum[key] = beta1*prev_bMomentum[key] + (1 - beta1) * acc_grad_bias[key]

                # update weight history
                for (key, value) in self.wHistory.items():
                    self.wHistory[key] = beta2 * prev_wHistory[key] + (1 - beta2) * acc_grad_weights[key] ** 2

                # update bias history
                for (key, value) in self.bHistory.items():
                    self.bHistory[key] = beta2 * prev_bHistory[key] + (1 - beta2) * acc_grad_bias[key] ** 2


                # compute intermediate values
                for (key, value) in self.weights.items():
                    wMomentum_hat[key] = self.wMomentum[key] / (1 - np.power(beta1, epoch + 1))
                
                for (key, value) in self.bias.items():
                    bMomentum_hat[key] = self.bMomentum[key] / (1 - np.power(beta1, epoch + 1))

                for (key, value) in self.weights.items():
                    wHistory_hat[key] = self.wHistory[key] / (1 - np.power(beta2, epoch + 1))

                for (key, value) in self.bias.items():
                    bHistory_hat[key] = self.bHistory[key] / (1 - np.power(beta2, epoch + 1))

                # update weights
                for (key, value) in self.weights.items():
                    num1 = ((self.learningRate / self.batch_size) / np.sqrt(wHistory_hat[key] + epsilon))
                    num2 = beta1 * wMomentum_hat[key] + ((1 - beta1) * acc_grad_weights[key] / (1 - beta1 ** (epoch + 1)))
                    self.weights[key] = self.weights[key] - num1*num2

                # update bias
                for (key, value) in self.bias.items():
                    num1 = ((self.learningRate / self.batch_size) / np.sqrt(bHistory_hat[key] + epsilon))
                    num2 = beta1 * bMomentum_hat[key] + ((1 - beta1) * acc_grad_bias[key] / (1 - beta1 ** (epoch + 1)))
                    self.bias[key] = self.bias[key] - num1*num2

                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
            
            # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:
            
            # update weight momentum
            for (key, value) in self.wMomentum.items():
                self.wMomentum[key] = beta1*prev_wMomentum[key] + (1 - beta1) * acc_grad_weights[key]

            # update bias momentum
            for (key, value) in self.bMomentum.items():
                self.bMomentum[key] = beta1*prev_bMomentum[key] + (1 - beta1) * acc_grad_bias[key]

            # update weight history
            for (key, value) in self.wHistory.items():
                self.wHistory[key] = beta2 * prev_wHistory[key] + (1 - beta2) * acc_grad_weights[key] ** 2

            # update bias history
            for (key, value) in self.bHistory.items():
                self.bHistory[key] = beta2 * prev_bHistory[key] + (1 - beta2) * acc_grad_bias[key] ** 2


            # compute intermediate values
            for (key, value) in self.weights.items():
                wMomentum_hat[key] = self.wMomentum[key] / (1 - np.power(beta1, epoch + 1))
            
            for (key, value) in self.bias.items():
                bMomentum_hat[key] = self.bMomentum[key] / (1 - np.power(beta1, epoch + 1))

            for (key, value) in self.weights.items():
                wHistory_hat[key] = self.wHistory[key] / (1 - np.power(beta2, epoch + 1))

            for (key, value) in self.bias.items():
                bHistory_hat[key] = self.bHistory[key] / (1 - np.power(beta2, epoch + 1))

            # update weights
            for (key, value) in self.weights.items():
                num1 = ((self.learningRate / self.batch_size) / np.sqrt(wHistory_hat[key] + epsilon))
                num2 = beta1 * wMomentum_hat[key] + ((1 - beta1) * acc_grad_weights[key] / (1 - beta1 ** (epoch + 1)))
                self.weights[key] = self.weights[key] - num1*num2

            # update bias
            for (key, value) in self.bias.items():
                num1 = ((self.learningRate / self.batch_size) / np.sqrt(bHistory_hat[key] + epsilon))
                num2 = beta1 * bMomentum_hat[key] + ((1 - beta1) * acc_grad_bias[key] / (1 - beta1 ** (epoch + 1)))
                self.bias[key] = self.bias[key] - num1*num2

            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()
        
                
        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()
        
        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # runs gradient descent for one epoch
    def oneEpochGD(self, epoch):
        ''' Executes A Single Epoch for Vanilla Gradient Descent Algorithm.
            Returns the training loss,training accuracy,validaiton loss and validation accuracy,averaged over all points. '''

        n = self.train_n_samples
        
        # randomizing batches
        idx = np.random.permutation(self.train_n_samples)
        self.x_train = self.x_train[idx]
        self.y_train = self.y_train[idx]

        input = self.x_train
        actual_y = self.y_train

        # Total Loss for epoch
        loss_input = 0
        count = 0

        # set accumalated gradients to zero
        acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # execute one epoch for all datapoints in train set
        for index in range(n):

            # perform forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # compute loss
            loss_input += self.loss_function(predicted_y, actual_y[index])

            # perofrm backward propagation
            grad_weights, grad_bias = self.backward_propagation(index, actual_y)

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

            # accumulate grad_weights and grad_bais for each input
            for (key, value) in grad_weights.items():
                acc_grad_weights[key] = acc_grad_weights[key] + \
                    grad_weights[key]

            for (key, value) in grad_bias.items():
                acc_grad_bias[key] = acc_grad_bias[key] + grad_bias[key]

            # update weights and bias if the number of datapoints in batch_size are divisble by batch_size
            if ((index + 1) % self.batch_size == 0):

                # update weights
                for (key, value) in self.weights.items():
                    self.weights[key] = self.weights[key] - ((self.learningRate / self.batch_size) * acc_grad_weights[key])
                
                # update bias
                for (key, value) in self.bias.items():
                    self.bias[key] = self.bias[key] - ((self.learningRate / self.batch_size) * acc_grad_bias[key])

                # set accumalated gradients to zero
                acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # if the number of datapoints in batch is not divisible by batch_size update weights and bias 
        if n % self.batch_size != 0:

            # update weights
            for (key, value) in self.weights.items():
                self.weights[key] = self.weights[key] - ((self.learningRate / self.batch_size) * acc_grad_weights[key])
            
            # update bias
            for (key, value) in self.bias.items():
                self.bias[key] = self.bias[key] - ((self.learningRate / self.batch_size) * acc_grad_bias[key])

            # set accumalated gradients to zero
            acc_grad_weights, acc_grad_bias = self.make_accumalate_zero()

        # compute trainAccuracy,trainLoss averaged over train size
        trainAccuracy = count / n
        trainLoss = loss_input / n + self.regularize()
        
        # compute validationAccuracy,validationLoss averaged over test size 
        validationLoss, validationAccuracy = self.computeTestLossAndAccuracy()
        
        return trainLoss, trainAccuracy, validationLoss, validationAccuracy

    # executes a single epoch of the FeedForward NN according to the optimizer function. 
    def executeOneEpoch(self,epoch):
        
        if self.optimizer == "sgd":
            return self.oneEpochSGD(epoch)

        if self.optimizer == "momentum":
            return self.oneEpochMOMENTUM(epoch)

        if self.optimizer == "nestrov":
            return self.oneEpochNAG(epoch)

        if self.optimizer == "rmsprop":
            return self.oneEpochRMSPROP(epoch)

        if self.optimizer == "adam":
            return self.oneEpochADAM(epoch)

        if self.optimizer == "nadam":
            return self.oneEpochNADAM(epoch)

        if self.optimizer == "gd":
            return self.oneEpochGD(epoch)

    # computes validation loss and validation accuracy 
    def computeTestLossAndAccuracy(self):
        
        validation_size = self.y_validate.shape[0]
        test_loss = 0
        count = 0

        input = self.x_validate
        actual_y = self.y_validate

        for index in range(0, validation_size):

            # perform forward propagation
            self.forward_propagation(input, index)
            predicted_y = self.post_activation["h" + str(self.L)]

            # compute loss
            test_loss += self.loss_function(predicted_y, actual_y[index])

            # compute the number of datapoints which are correctly classified
            indexWithMaxProb = np.argmax(predicted_y)
            if (actual_y[index] == (indexWithMaxProb)):
                count = count + 1

        # compute validationAccuracy,validationLoss averaged over validation size 
        validationAccuracy = count / validation_size
        validationLoss = test_loss / validation_size + self.regularize()
        
        return validationLoss, validationAccuracy

    '''<----------------------------Question 1------------------------------------->'''
    def question_1(self):

        wandb.init(
                # set the wandb project where this run will be logged
                project = feed_forward.parameters["wandb_project"],
                # config = sweep_config
        )

        # dictionary of labels to be added
        labels_added = {}

        ''' 
        Running the loop for the number of training samples.
        In each iteration,a random index is generated and we extract the feature and label at the generated index.
        If the label is already in the labels_added dictionary,we ignore that label,else we add that (label,feature) 
        as (key,value) pair in dictionary (so that one label is considered only once).
        '''
        images = []
        for i in range(self.train_n_samples):
            index = random.randrange(self.train_n_samples)
            feature = self. x_train[index]
            label = self.y_train[index]
            if (label in labels_added.keys()):
                continue
            labels_added[label] = feature
            image = wandb.Image(
                labels_added[label], caption=f"{self.title[label]}")
            images.append(image)
        wandb.log({"Images": images})
    
    '''<----------------------------Question 2------------------------>'''

    def feed_forward_q2(self): 
        # initialization of weights
        self.weightsAndBiasInitializer()
 
        # train the data
        for i in range(1, self.epoch + 1):
            (train_Loss, train_Accuracy, validation_Loss,
             validation_Accuracy) = self.executeOneEpoch(i)
            
        # generating the random index to test the model and finding the y for that
        index = np.random.randint(self.y_validate.shape[0])
        input = self.x_validate
        self.forward_propagation(input, index)
        predicted_y = self.post_activation["h" + str(self.L)]
        print(predicted_y)

    '''<----------------------------Question 3-4------------------------------------->'''
    def feed_forward_q3_4(self):
        
        self.weights = dict()
        self.bias = dict()
        self.wHistory = dict()
        self.bHistory = dict()
        self.wMomentum = dict()
        self.bMomentum = dict()
        
        # initialization of weights and bias
        self.weightsAndBiasInitializer()

        # initializing history for weights and bias
        self.historyInitializer()

        # initializing momentum for weights and bias
        self.momentumInitializer()

        self.validation_Accuracy = 0


        # run feedforward NN 
        for i in range(1, self.epoch + 1):
              (train_Loss, train_Accuracy, validation_Loss,self.validation_Accuracy) = self.executeOneEpoch(i)
              print("epoch:{epoch}, train loss:{train_l}, train accuracy:{train_ac}, validation loss:{validation_l}, validation accuracy:{validation_ac}".\
                  format(epoch = i,train_l = train_Loss,train_ac = train_Accuracy,validation_l = validation_Loss,validation_ac = self.validation_Accuracy))
            
              wandb.log({'train loss':train_Loss, 'train accuracy':train_Accuracy,'validation loss':validation_Loss, 'validation accuracy':self.validation_Accuracy})
            

In [2]:
feed_forward = FeedForward()


In [3]:
sweep_config = {

        'method' : 'random', #grid ,random - generates exponential ways,bayesian  efficient way
        'name' : 'random_sweep mse',
        'metric' : {
            'name' : 'validation accuracy',
            'goal' : 'maximize'
        },
        'parameters':{
                'epochs' : {
                    'values' : [5,10]
                },
                'number_of_hidden_layer':{
                    'values' : [3,4,5]
                },
                'size_of_hidden_layer' : {
                    'values' :[32,64,128]
                },
                'weight_decay' : {
                    'values' : [0,0.0005,0.5]
                },
                'learning_rate' : {
                    'values' : [1e-3,1e-4]
                },
                'optimizer' : {
                    'values' : ['sgd','momentum','nestrov','rmsprop','adam','nadam']
                },
                'batch_size' : {
                        'values' : [16,32,64]
                },
                'weight_initialization' :{
                    'values' : ['random','Xavier']
                },
                'activation' : {
                    'values' : ['sigmoid','tanh','ReLU']
                }
        }
}
sweep_id = wandb.sweep(sweep = sweep_config,project= feed_forward.parameters["wandb_project"])

Create sweep with ID: hl8fpi30
Sweep URL: https://wandb.ai/cs22m019/DL%20Final%20Assignment%201/sweeps/hl8fpi30


In [4]:
def train():
    wandb.init(
                # set the wandb project where this run will be logged
                # project = feed_forward.parameters["wandb_project"],
                config = sweep_config
    )
    
    feed_forward.epoch = wandb.config.epochs
    feed_forward.nnl = wandb.config.size_of_hidden_layer
    feed_forward.weightDecay =  wandb.config.weight_decay
    feed_forward.learningRate = wandb.config.learning_rate
    feed_forward.optimizer = wandb.config.optimizer
    feed_forward.batch_size = wandb.config.batch_size
    feed_forward.weightInitialization = wandb.config.weight_initialization
    feed_forward.activationFunction = wandb.config.activation
    feed_forward.L = wandb.config.number_of_hidden_layer + 1
    feed_forward.weight_decay = wandb.config.weight_decay


    wandb.run.name = "optimizer_" + str(wandb.config.optimizer) +  "_hl_"+ str(wandb.config.number_of_hidden_layer) + "_bs_" + str(wandb.config.batch_size) + "_ac_" + str(wandb.config.activation)    
    feed_forward.feed_forward_q3_4()

In [ ]:
wandb.agent(sweep_id=sweep_id,function = train,count = 100)

wandb: Agent Starting Run: bdrj5ovc with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	number_of_hidden_layer: 4
wandb: 	optimizer: nestrov
wandb: 	size_of_hidden_layer: 32
wandb: 	weight_decay: 0
wandb: 	weight_initialization: Xavier
wandb: Currently logged in as: cs22m019. Use `wandb login --relogin` to force relogin


epoch:1, train loss:0.927165388652894, train accuracy:0.10031481481481481, validation loss:0.9161960841437223, validation accuracy:0.09983333333333333
epoch:2, train loss:0.9124516756557705, train accuracy:0.10337037037037038, validation loss:0.9088522957948203, validation accuracy:0.09983333333333333
epoch:3, train loss:0.9074925278220668, train accuracy:0.11040740740740741, validation loss:0.9057520745352943, validation accuracy:0.15483333333333332
epoch:4, train loss:0.9051262502624003, train accuracy:0.11212962962962963, validation loss:0.9040933839838856, validation accuracy:0.10066666666666667
epoch:5, train loss:0.9036505071742047, train accuracy:0.12496296296296296, validation loss:0.9028726372289254, validation accuracy:0.136


train accuracy,▁▂▄▄█
train loss,█▄▂▁▁
validation accuracy,▁▁█▁▆
validation loss,█▄▃▂▁
train accuracy,0.12496
train loss,0.90365
validation accuracy,0.136
validation loss,0.90287


wandb: Agent Starting Run: emcr40b3 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	number_of_hidden_layer: 5
wandb: 	optimizer: adam
wandb: 	size_of_hidden_layer: 64
wandb: 	weight_decay: 0
wandb: 	weight_initialization: Xavier


epoch:1, train loss:0.8216885556579229, train accuracy:0.3443703703703704, validation loss:0.7279213011246519, validation accuracy:0.48783333333333334
epoch:2, train loss:0.6707726320160337, train accuracy:0.5514814814814815, validation loss:0.6140853886532305, validation accuracy:0.6071666666666666
epoch:3, train loss:0.577303876778874, train accuracy:0.6468518518518519, validation loss:0.5354249577767946, validation accuracy:0.6731666666666667
epoch:4, train loss:0.5094832336615671, train accuracy:0.6886666666666666, validation loss:0.47765335662142716, validation accuracy:0.7033333333333334
epoch:5, train loss:0.45917963716616883, train accuracy:0.7144259259259259, validation loss:0.4348375755536872, validation accuracy:0.732
epoch:6, train loss:0.42174869358847916, train accuracy:0.7370740740740741, validation loss:0.4032720700396119, validation accuracy:0.7531666666666667
epoch:7, train loss:0.393227069619491, train accuracy:0.7524259259259259, validation loss:0.3787522558797738, 

...

train accuracy,▁▁▁▁▁
train loss,█▅▃▂▁
validation accuracy,▁▁▁▁▁
validation loss,█▅▃▂▁
train accuracy,0.09972
train loss,0.93324
validation accuracy,0.1025
validation loss,0.93129


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: x9agak2l with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	number_of_hidden_layer: 5
wandb: 	optimizer: momentum
wandb: 	size_of_hidden_layer: 128
wandb: 	weight_decay: 0
wandb: 	weight_initialization: Xavier


epoch:1, train loss:0.906267604467005, train accuracy:0.10033333333333333, validation loss:0.9007645842325842, validation accuracy:0.09433333333333334
epoch:2, train loss:0.9001554587896127, train accuracy:0.0976111111111111, validation loss:0.9000985791510114, validation accuracy:0.10066666666666667
epoch:3, train loss:0.9000509055759877, train accuracy:0.09464814814814815, validation loss:0.9000936920367884, validation accuracy:0.09433333333333334
epoch:4, train loss:0.9000563081556443, train accuracy:0.09651851851851852, validation loss:0.9000474187915609, validation accuracy:0.09433333333333334
epoch:5, train loss:0.9000486991007915, train accuracy:0.09603703703703703, validation loss:0.9000642453225541, validation accuracy:0.0915
epoch:6, train loss:0.900047597351778, train accuracy:0.09759259259259259, validation loss:0.9000494387176233, validation accuracy:0.10066666666666667
epoch:7, train loss:0.9000443192882487, train accuracy:0.09651851851851852, validation loss:0.9000315820

train accuracy,█▅▁▃▃▅▃▂▃▄
train loss,█▁▁▁▁▁▁▁▁▁
validation accuracy,▅▆▅▅▅▆▆▆▁█
validation loss,█▂▂▁▁▁▁▁▁▁
train accuracy,0.0973
train loss,0.90005
validation accuracy,0.113
validation loss,0.90003


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: sj16ge60 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	number_of_hidden_layer: 5
wandb: 	optimizer: nestrov
wandb: 	size_of_hidden_layer: 128
wandb: 	weight_decay: 0
wandb: 	weight_initialization: random
